In [10]:
from nqueen import NQueen, get_queensidxs
import math
import random
import time

In [11]:
def neighborhood(nqueen): # Retorna a lista de vizinhos, que são os possíveis movimentos para a rainha
    nb = []  # Lista para armazenar os vizinhos
    queens = get_queensidxs(nqueen.board)  # Obtém as posições das rainhas no tabuleiro
    for i in range(nqueen.n):
        p = queens[i]  # Obtém a posição atual da rainha
        for j in range(nqueen.n):
            if j != p:  # Verifica se a coluna é diferente da posição atual da rainha
                cq = nqueen.deepcopy()  # Cria uma cópia do estado atual do tabuleiro
                cq.moveTo(j, i)  # Move a rainha para a nova coluna
                nb.append(cq)  # Adiciona o novo tabuleiro à lista de vizinhos
    return nb  # Retorna a lista de vizinhos


In [12]:
def hillclimbing(nqueen):
    while True:
        candidates = neighborhood(nqueen)  # Obtém a lista de vizinhos possíveis da configuração atual
        betterValue = nqueen.attacks()  # Obtém o número de ataques da configuração atual
        betterNeighbor = None  # Variável para armazenar o melhor vizinho encontrado
        for candidate in candidates:
            v = candidate.attacks()  # Obtém o número de ataques do vizinho atual
            if v < betterValue:  # Verifica se o vizinho atual tem menos ataques
                betterNeighbor = candidate  # Atualiza o melhor vizinho encontrado
                betterValue = v  # Atualiza o número de ataques do melhor vizinho
        
        if betterNeighbor is None:  # Se não houver um vizinho melhor
            return nqueen  # Retorna a configuração atual (ótimo local)
        else:
            nqueen = betterNeighbor  # Atualiza a configuração atual para o melhor vizinho encontrado


In [13]:
def hillclimbing_steepest_ascent(nqueen):
    while True:
        current_attacks = nqueen.attacks()  # Obtem o número de ataques da configuração atual
        candidates = neighborhood(nqueen)  # Gera candidatos vizinhos
        best_neighbor = None  # Inicializa o melhor vizinho
        best_attacks = current_attacks  # Inicializar o número mínimo de ataques
        for candidate in candidates:
            candidate_attacks = candidate.attacks()  # Calcula o número de ataques do candidato
            if candidate_attacks < best_attacks:  # Verificar se o candidato tem menos ataques do que o melhor vizinho atual
                best_neighbor = candidate  # Atualiza o melhor vizinho
                best_attacks = candidate_attacks  # Atualiza o número mínimo de ataques
        # Verificar se não há melhor vizinho ou se o número mínimo de ataques não diminuiu
        if best_neighbor is None or best_attacks >= current_attacks:  
            return nqueen  # Retornar a configuração atual
        else:
            nqueen = best_neighbor  # Atualizar a configuração atual com o melhor vizinho


In [14]:
def hillclimbing_first_choice(nqueen):
    while True:
        current_attacks = nqueen.attacks()  # Obtem o número de ataques da configuração atual
        best_neighbor = None

        # Itera pelo número de rainhas no tabuleiro
        for _ in range(len(nqueen.board)):
            candidate = nqueen.deepcopy()  # Cria uma cópia da configuração atual
            candidate.moveTo(random.randint(0, nqueen.n-1), random.randint(0, nqueen.n-1))  # Move uma rainha aleatoriamente
            candidate_attacks = candidate.attacks()  # Obtem o número de ataques na configuração candidata

            if candidate_attacks < current_attacks:  # Se o número de ataques na configuração candidata for menor
                best_neighbor = candidate  # Atualizar o melhor vizinho encontrado
                break  # Parar a busca

        if best_neighbor is None:  # Se nenhum vizinho melhor foi encontrado
            return nqueen  # Retornar a configuração atual
        else:
            nqueen = best_neighbor  # Atualizar a configuração atual para o melhor vizinho encontrado


In [15]:
def hillclimbing_random_restarts(nqueen, max_restarts = 25):
    best_solution = None  # Inicializa a melhor solução encontrada
    for restart in range(max_restarts):
        current = nqueen.deepcopy()  # Cria uma cópia do estado atual
        current = hillclimbing(current)  # Aplica o algoritmo Hill Climbing no estado atual
        if (best_solution is None) or (current.attacks() < best_solution.attacks()):
            # Se é a primeira solução ou a solução atual é melhor que a melhor solução anterior
            best_solution = current  # Atualiza a melhor solução
        if best_solution.attacks() == 0:
            break # Se encontrou uma solução ideal, interrompe a execução
            
    return best_solution


In [16]:
def simulated_annealing(nqueen, initial_temperature=20, final_temperature=0.1, alpha=0.5):
    # Inicialização dos estados e energia atual
    current_state = nqueen  # Estado atual
    current_energy = current_state.attacks()  # Energia atual
    best_state = current_state  # Melhor estado encontrado até o momento
    best_energy = current_energy  # Melhor energia encontrada até o momento
    
    # Configuração inicial da temperatura e limite de iterações
    temperature = initial_temperature  # Temperatura inicial
    
    # Loop principal do simulated annealing
    while temperature > final_temperature:
        
        # Escolha de um vizinho aleatório
        neighbor = neighborhood(current_state)[random.randint(0, len(neighborhood(current_state)) - 1)]
        neighbor_energy = neighbor.attacks()  # Energia do vizinho escolhido
        
        # Cálculo da variação de energia
        energy_delta = neighbor_energy - current_energy
        
        # Critério de aceitação do novo estado
        # Representa a probabilidade de aceitar um estado pior (com energia mais alta) como uma forma de escapar de mínimos locais
        if (energy_delta < 0 or random.random()) < (math.exp(-energy_delta / temperature)):
            current_state = neighbor
            current_energy = neighbor_energy
        
        # Atualização do melhor estado encontrado
        if current_energy < best_energy:
            best_state = current_state
            best_energy = current_energy
        
        # Atualização da temperatura
        temperature *= alpha
    
    return best_state


In [17]:
def teste(funcao, verbose=False):
    tempo = time.time()
    print(funcao.__name__)
    counter_mean = 0  
    success = 0
    for i in range(maxit):
        counter = 0  
        while True:
            current = NQueen(8)  
            current = funcao(current)  
            counter += 1     
            if( current.attacks() <= 0):
                success += 1
                break  
        if( i == maxit-1 and verbose):
            print(current)         
        counter_mean += counter  

    counter_mean /= maxit  
    print("Tentativas: ", counter_mean)   
    print("Sucessos: ", success)
    tempo = time.time() - tempo
    print("Custo: %.2f segundos (%2.2f minutos)\n"%(tempo, tempo/60) )   

In [18]:
maxit = 100
teste(hillclimbing)
teste(hillclimbing_steepest_ascent)
teste(hillclimbing_first_choice)
teste(hillclimbing_random_restarts)
teste(simulated_annealing)


hillclimbing
Tentativas:  6.43
Sucessos:  100
Custo: 43.35 segundos (0.72 minutos)

hillclimbing_steepest_ascent
Tentativas:  7.06
Sucessos:  100
Custo: 43.28 segundos (0.72 minutos)

hillclimbing_first_choice
Tentativas:  371.83
Sucessos:  100
Custo: 137.65 segundos (2.29 minutos)

hillclimbing_random_restarts
Tentativas:  5.88
Sucessos:  100
Custo: 628.93 segundos (10.48 minutos)

simulated_annealing


KeyboardInterrupt: 